In [ ]:
# Cell 1 - Setup: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os

TELNET_DATADIR = '/content/drive/MyDrive/telnet_data'

# Verify required paths exist
virtual_stores_path = os.path.join(TELNET_DATADIR, 'virtual_stores')
final_feats_path = os.path.join(TELNET_DATADIR, 'data/models/final_feats.txt')

missing_paths = []
if not os.path.exists(virtual_stores_path):
    missing_paths.append(virtual_stores_path)
if not os.path.exists(final_feats_path):
    missing_paths.append(final_feats_path)

if missing_paths:
    print("ERROR: Required paths are missing!")
    print("Please run prepare_local_data.py locally first.")
    for path in missing_paths:
        print(f"  Missing: {path}")
else:
    os.environ['TELNET_DATADIR'] = TELNET_DATADIR
    print(f"Data directory set: {TELNET_DATADIR}")
    print("All required paths verified!")

In [ ]:
# Cell 2 - Install Dependencies
%cd /content
!rm -rf telnet
!git clone https://github.com/gscerveira/telnet.git
%cd telnet

# Install uv for faster package management
!pip install -q uv

# Install dependencies with uv
!uv pip install --system -q -r docker/requirements.txt
!uv pip install --system -q s3fs geopandas rioxarray icechunk virtualizarr obstore

print("Dependencies installed!")

In [ ]:
# Cell 3 - Verify GPU
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: No GPU available! GPU-intensive operations will be slow.")

In [ ]:
# Cell 4 - Model Selection (GPU)
%cd /content/telnet
import os
os.environ['TELNET_DATADIR'] = '/content/drive/MyDrive/telnet_data'

!chmod +x model_selection.sh
!./model_selection.sh 100 1

In [ ]:
# Cell 5 - Model Testing (GPU)
%cd /content/telnet
import os
os.environ['TELNET_DATADIR'] = '/content/drive/MyDrive/telnet_data'

!python model_testing.py -n 100 -c 1

In [ ]:
# Cell 6 - Generate Forecasts (GPU)
%cd /content/telnet
import os
os.environ['TELNET_DATADIR'] = '/content/drive/MyDrive/telnet_data'

!chmod +x generate_forecast.sh

init_dates = ['202501', '202504', '202507', '202510']

for init_date in init_dates:
    print(f"\n=== Generating forecast for {init_date} ===")
    !./generate_forecast.sh {init_date} 1

In [ ]:
# Cell 7 - Extract Maranhao Region
%cd /content/telnet
import os
import glob

os.environ['TELNET_DATADIR'] = '/content/drive/MyDrive/telnet_data'
DATADIR = os.environ['TELNET_DATADIR']

init_dates = ['202501', '202504', '202507', '202510']

for init_date in init_dates:
    results_dir = f'{DATADIR}/results/{init_date}'
    if os.path.exists(results_dir):
        for f in glob.glob(f'{results_dir}/*.nc'):
            basename = os.path.basename(f)
            if not basename.startswith('maranhao_'):
                output = f'{results_dir}/maranhao_{basename}'
                !python extract_maranhao.py "{f}" "{output}" --shapefile-dir {DATADIR}/shapefiles
                print(f"Extracted: {output}")

In [ ]:
# Cell 8 - View Results Summary
import os

DATADIR = '/content/drive/MyDrive/telnet_data'
init_dates = ['202501', '202504', '202507', '202510']

print("=" * 50)
print("GPU-Only Workflow Complete!")
print("=" * 50)

for init_date in init_dates:
    results_dir = f'{DATADIR}/results/{init_date}'
    if os.path.exists(results_dir):
        file_count = len([f for f in os.listdir(results_dir) if os.path.isfile(os.path.join(results_dir, f))])
        print(f"{init_date}: {file_count} files in {results_dir}")
    else:
        print(f"{init_date}: Directory not found")